In [2]:
import pandas as pd


df = pd.read_csv('relative_path_to_csv')

# Ensure all parts are converted to string before concatenation
df['company_id'] = pd.factorize(df['company_name'].astype(str) + df['address'].astype(str) + df['city'].astype(str) + df['state'].astype(str) + df['zip'].astype(str))[0] + 1
df['department_id'] = pd.factorize(df['department'].astype(str))[0] + 1


# Create DataFrames for Company, Department, and Employee tables
df_company = df[['company_id', 'company_name', 'address', 'city', 'state', 'zip']].drop_duplicates()
df_department = df[['department_id', 'department']].drop_duplicates().rename(columns={'department': 'department_name'})
df_employee = df[['first_name', 'last_name', 'phone1', 'phone2', 'email', 'company_id', 'department_id']]


In [3]:
import psycopg2
from psycopg2 import extras

try:
    connection = psycopg2.connect(
        dbname='postgres',
        user='postgres',
        password='example',
        host='localhost'
    )
except psycopg2.Error as e:
    print("Error: Could not make connection to the PostgreSQL database")
    print(e)


In [4]:
def insert_into_table(connection, df, table_name):
    """
    Insert DataFrame into the specified table using a connection.
    """
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    query = f"INSERT INTO {table_name} ({cols}) VALUES %s ON CONFLICT DO NOTHING"
    cursor = connection.cursor()
    try:
        extras.execute_values(cursor, query, tuples, template=None)
        connection.commit()
    except psycopg2.Error as e:
        print(f"Error: Could not insert data into {table_name}")
        print(e)
    finally:
        cursor.close()

# Insert data into each table
insert_into_table(connection, df_company, 'Company')
insert_into_table(connection, df_department, 'Department')
insert_into_table(connection, df_employee, 'Employee')
